In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yaml
import os
import urlparse
import requests
import json

from synorchestrator.trs_client.client import TRSClient
from synorchestrator.wes_client.client import WESClient
from synorchestrator.util import build_wes_request

DEBUG:rdflib:RDFLib Version: 4.2.2


## Set up TRS and WES clients

Load options and paramters for TRS and WES servers from local configs.

In [3]:
with open('../toolregistries.config', 'rb') as f:
    trs_config = yaml.load(f)
    
with open('../workflowservices.config', 'rb') as f:
    wes_config = yaml.load(f)

In [4]:
tclient = TRSClient(**trs_config['dockstore'])
wclient = WESClient(**wes_config['arvados-wes'])

## Working with TRS

### Get checker workflow information

In [5]:
workflow_id = 'github.com/dockstore-testing/md5sum-checker'

#### Get checker for specified workflow

In [6]:
checker_workflow = tclient.get_workflow_checker(workflow_id)
checker_workflow['id']

DEBUG:root:retrieving workflow entry from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker HTTP/1.1" 200 418
DEBUG:synorchestrator.trs_client.client:found checker workflow: github.com/dockstore-testing/md5sum-checker/_cwl_checker
DEBUG:root:retrieving workflow entry from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker HTTP/1.1" 200 496


u'#workflow/github.com/dockstore-testing/md5sum-checker/_cwl_checker'

#### Get checker descriptor for specified version and workflow type

In [7]:
checker_descriptor = tclient.get_workflow_descriptor(checker_workflow['id'], 'develop', 'CWL')
checker_descriptor

DEBUG:synorchestrator.trs_client.client:getting descriptor from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/descriptor
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/descriptor HTTP/1.1" 200 541


{u'descriptor': u'cwlVersion: v1.0\nclass: Workflow\n\nrequirements:\n  - class: SubworkflowFeatureRequirement\n\n#dct:creator:\n#  \'@id\': http://orcid.org/0000-0002-7681-6415\n#  foaf:name: Brian O\'Connor\n#  foaf:mbox: mailto:briandoconnor@gmail.com\n\n#dct:contributor:\n#  foaf:name: Denis Yuen\n#  foaf:mbox: mailto:denis.yuen@oicr.on.ca\n\ninputs:\n  input_file: File\n  expected_md5: string\n\noutputs:\n  workflow_output_file:\n    type: File\n    outputSource: checker/results_file\n\nsteps:\n  md5sum:\n    run: md5sum/md5sum-workflow.cwl\n    in:\n      input_file: input_file\n    out: [output_file]\n  checker:\n    run: checker/md5sum-checker.cwl\n    in:\n      input_file: md5sum/output_file\n      expected_md5: expected_md5\n    out: [results_file]\n\ndoc: |\n  This demonstrates how to wrap a "real" tool with a checker workflow that runs both the tool and a tool that performs verification of results\n',
 u'type': u'CWL',
 u'url': u'https://raw.githubusercontent.com/dockstore

#### Get checker test params for specified version and workflow type

In [8]:
checker_tests = tclient.get_workflow_tests(checker_workflow['id'], 'develop', 'CWL')
checker_tests

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/tests HTTP/1.1" 200 206
DEBUG:synorchestrator.trs_client.client:getting descriptor from tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/descriptor
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dockstore.org
DEBUG:urllib3.connectionpool:https://dockstore.org:8443 "GET /api/ga4gh/v2/tools/%23workflow%2Fgithub.com%2Fdockstore-testing%2Fmd5sum-checker%2F_cwl_checker/versions/develop/CWL/descriptor HTTP/1.1" 200 541


[{u'test': u'{\n  "input_file": {\n        "class": "File",\n        "path": "md5sum.input"\n    },\n    "expected_md5": "00579a00e3e7fa0674428ac7049423e2"\n}\n',
  u'url': u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-input-cwl.json'}]

## Prepare workflow run request

### Build request

In [10]:
workflow_request = build_wes_request(
    checker_descriptor['url'], checker_tests[0]['url']
)
workflow_request

DEBUG:cachecontrol.controller:Looking up "https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-input-cwl.json" in the cache
DEBUG:cachecontrol.controller:Current age based on date: 376
DEBUG:cachecontrol.controller:Freshness lifetime from max-age: 300
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /dockstore-testing/md5sum-checker/develop/checker-input-cwl.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /dockstore-testing/md5sum-checker/develop/md5sum.input HTTP/1.1" 200 0


{'workflow_params': ordereddict([('input_file', ordereddict([('class', 'File'), ('location', u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/md5sum.input')])), ('expected_md5', '00579a00e3e7fa0674428ac7049423e2')]),
 'workflow_type': 'CWL',
 'workflow_type_version': 'v1.0',
 'workflow_url': u'https://raw.githubusercontent.com/dockstore-testing/md5sum-checker/develop/checker-workflow-wrapping-workflow.cwl'}

## Working with WES

### Verify WES connection is working

In [11]:
wclient.get_service_info()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/service-info HTTP/1.1" 200 325


{u'engine_versions': u'cwl-runner',
 u'key_values': {},
 u'supported_filesystem_protocols': [u'file', u'http', u'https', u'keep'],
 u'supported_wes_versions': u'0.2.1',
 u'system_state_counts': {},
 u'workflow_type_versions': {u'CWL': {u'workflow_type_version': [u'v1.0']}}}

### Run a checker workflow

#### Submit new workflow run

In [12]:
r = wclient.run_workflow(workflow_request)
r

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "POST /ga4gh/wes/v1/workflows HTTP/1.1" 200 51


{u'workflow_id': u'qr1hi-xvhdp-dgj0pct9ifm4aiq'}

#### Monitor workflow run status

In [13]:
r = wclient.get_workflow_run_status(r['workflow_id'])
while r['state'] in ('QUEUED', 'INITIALIZING', 'RUNNING'):
    time.sleep(1)
    r = wclient.get_workflow_run_status(r['workflow_id'])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/workflows/qr1hi-xvhdp-dgj0pct9ifm4aiq/status HTTP/1.1" 200 74


#### Check workflow run details

In [14]:
wclient.get_workflow_run(r['workflow_id'])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): wes.qr1hi.arvadosapi.com
DEBUG:urllib3.connectionpool:https://wes.qr1hi.arvadosapi.com:443 "GET /ga4gh/wes/v1/workflows/qr1hi-xvhdp-dgj0pct9ifm4aiq HTTP/1.1" 200 2384


{u'outputs': {u'workflow_output_file': {u'basename': u'results.json',
   u'class': u'File',
   u'location': u'https://download.qr1hi.arvadosapi.com/c=51a1c8291787268318b8e84da2371019+117/_/results.json',
   u'size': 57}},
 u'request': {},
 u'state': u'COMPLETE',
 u'task_logs': [],
 u'workflow_id': u'qr1hi-xvhdp-dgj0pct9ifm4aiq',
 u'workflow_log': {u'cmd': [u''],
  u'endTime': u'',
  u'exit_code': 0,
  u'startTime': u'',
  u'stderr': u'2018-05-15T15:20:10.578578670Z cwltool INFO: /usr/local/bin/arvados-cwl-runner 1.1.4.20180511203654, arvados-python-client 1.1.4.20180510153813, cwltool 1.0.20180508202931\n2018-05-15T15:20:10.580624270Z cwltool INFO: Resolved \'/var/lib/cwl/workflow.json#main\' to \'file:///var/lib/cwl/workflow.json#main\'\n2018-05-15T15:20:12.682419170Z cwltool INFO: [workflow workflow.json#main] start\n2018-05-15T15:20:12.683911770Z cwltool INFO: [step md5sum] start\n2018-05-15T15:20:12.686741870Z cwltool INFO: [workflow md5sum] start\n2018-05-15T15:20:12.688149770Z cw